In [8]:
import sys
import os

# Add the 'src' folder to sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

os.chdir('D:\Python\Industry level\Deep learning\Deep-learning-Disease-Classifier')
%pwd

'D:\\Python\\Industry level\\Deep learning\\Deep-learning-Disease-Classifier'

In [9]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/AnkitSangode/Deep-learning-Disease-Classifier.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "AnkitSangode"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "391b761f20f4883ca8292a3e48b1b69ab2d166e7"

In [10]:
import tensorflow as tf

In [11]:
model = tf.keras.models.load_model('artifacts/training/trained_model.keras')

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
from diseaseClassifier.constants import *
from diseaseClassifier.utils.common import read_yaml,create_directories,save_json

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH

    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/trained_model.keras",
            training_data= "artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri= "https://dagshub.com/AnkitSangode/Deep-learning-Disease-Classifier.mlflow",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )

        return eval_config

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlpar

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset= "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("score.json"),data= scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy": self.score[1]}
            )

            # 1. Save locally
            self.model.save(self.config.path_of_model)  # creates a directory

            # 2. Log directory as artifact
            mlflow.log_artifacts(self.config.path_of_model, artifact_path="model")


In [29]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-02 05:43:50,147: INFO: yaml file: config\config.yaml loaded successfully]
[2025-08-02 05:43:50,159: INFO: yaml file: params.yaml loaded successfully]
[2025-08-02 05:43:50,165: INFO: Created directory at: artifacts]
Found 139 images belonging to 2 classes.


d:\Python\Industry level\Deep learning\Deep-learning-Disease-Classifier\kidney\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 45s 5s/step - accuracy: 0.5036 - loss: 6.0356
[2025-08-02 05:44:37,066: INFO: json file saved at: score.json]
🏃 View run masked-sheep-970 at: https://dagshub.com/AnkitSangode/Deep-learning-Disease-Classifier.mlflow/#/experiments/0/runs/418067858af742d7950a6ecfe1103f66
🧪 View experiment at: https://dagshub.com/AnkitSangode/Deep-learning-Disease-Classifier.mlflow/#/experiments/0
